In [1]:
import os
os.chdir("../")

In [2]:
import glob

mscx_path_list = []
for item in glob.glob("./xmls/mscx/*"):
    mscx_path_list.append(os.path.abspath(item))
mscx_path_list.sort()

tstree_path_list = []
for item in glob.glob("./xmls/tstree/*"):
    tstree_path_list.append(os.path.abspath(item))
tstree_path_list.sort()

In [3]:
from alm.comparator import tree_similarity_calculator as TSCalculator

tree_similarities = TSCalculator.calc_tree_similarities(mscx_path_list, tstree_path_list)

In [4]:
import pprint

tree_similarities_dict = {}
for item in tree_similarities:
    song_name = item.section_name[:-2]
    section = item.section_name[-1]

    if song_name not in tree_similarities_dict:
        tree_similarities_dict[song_name] = [-1, -1, -1, -1]

    if section == "A":
        tree_similarities_dict[song_name][0] = item.denominator
        tree_similarities_dict[song_name][1] = item.numerator
    elif section == "S":
        tree_similarities_dict[song_name][2] = item.denominator
        tree_similarities_dict[song_name][3] = item.numerator

res = [] 
for key, value in tree_similarities_dict.items():
    value.insert(0, key)
    res.append(value)

pprint.pprint(res)

[['BE_FREE', 45, 10, 53, 9],
 ['SUN_SHINE!!!', 99, 15, 58, 8],
 ['beautiful_days', 51, 11, 60, 8],
 ['no_more_war', 121, 18, 33, 3],
 ['たけてん', 51, 10, 77, 10],
 ['オレンジ', 75, 6, 114, 30],
 ['キセキ', 54, 8, 81, 13],
 ['サヨナラから始めよう', 112, 13, 36, 2],
 ['ビリーヴ', 74, 22, 82, 16],
 ['ルーキーズ', 105, 19, 101, 16],
 ['人', 111, 15, 77, 9],
 ['刹那', 47, 3, 48, 4],
 ['君想い', 109, 18, 71, 9],
 ['愛唄', 74, 12, 57, 13],
 ['扉', 87, 18, 54, 10],
 ['旅立ち', 57, 12, 58, 10],
 ['星影のエール', 58, 12, 65, 9],
 ['涙空', 113, 9, 29, 8],
 ['花唄', 87, 18, 55, 13],
 ['遥か', 95, 21, 63, 10]]
